# Trabalho Prático 02 - IA_24/25

## Autores:
- Nuno Castro_4944  
- Jorge Mendes_18466  
- André Freitas_25975  

### Janeiro 2025


## Introdução
A aplicação de técnicas de aprendizagem máquina e análise de dados em problemas práticos requer uma abordagem estruturada e bem planeada. Neste trabalho, exploraremos três temas principais: Classificação Automática, Agrupamento (Clustering) e Regras de Associação. Estes tópicos cobrem uma ampla gama de aplicações no campo da Inteligência Artificial e fornecem ferramentas fundamentais para analisar, prever e descobrir padrões em dados.

***





# Classificação Automática
A classificação automática é uma técnica de aprendizagem supervisionada em que os modelos são treinados para categorizar instâncias de dados em classes predefinidas.





### Proposta:
#### Classificação de imóveis como "Acessível", "Média" ou "Luxuosa"










## 1.  Seleção de dados





### 1.  Download do Dataset

Origem: https://www.kaggle.com/datasets/dansbecker/melbourne-housing-snapshot

In [1]:
# Download dataset from Kaggle
import kagglehub

dataset_path = "dansbecker/melbourne-housing-snapshot"
!kaggle datasets download {dataset_path} -p /content/ --unzip

Dataset URL: https://www.kaggle.com/datasets/dansbecker/melbourne-housing-snapshot
License(s): CC-BY-NC-SA-4.0
  0% 0.00/451k [00:00<?, ?B/s]
100% 451k/451k [00:00<00:00, 31.1MB/s]


### 1.1 Apresentação do Conteudo do ficheiro

In [2]:
# Imports
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt


# Read the file
path = "/content/melb_data.csv"
sales_data = pd.read_csv(path)
# Print head
sales_data.tail()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
13575,Wheelers Hill,12 Strada Cr,4,h,1245000.0,S,Barry,26/08/2017,16.7,3150.0,...,2.0,2.0,652.0,NaN,1981.0,NaN,-37.90562,145.16761,South-Eastern Metropolitan,7392.0
13576,Williamstown,77 Merrett Dr,3,h,1031000.0,SP,Williams,26/08/2017,6.8,3016.0,...,2.0,2.0,333.0,133.0,1995.0,NaN,-37.85927,144.87904,Western Metropolitan,6380.0
13577,Williamstown,83 Power St,3,h,1170000.0,S,Raine,26/08/2017,6.8,3016.0,...,2.0,4.0,436.0,NaN,1997.0,NaN,-37.85274,144.88738,Western Metropolitan,6380.0
13578,Williamstown,96 Verdon St,4,h,2500000.0,PI,Sweeney,26/08/2017,6.8,3016.0,...,1.0,5.0,866.0,157.0,1920.0,NaN,-37.85908,144.89299,Western Metropolitan,6380.0
13579,Yarraville,6 Agnes St,4,h,1285000.0,SP,Village,26/08/2017,6.3,3013.0,...,1.0,1.0,362.0,112.0,1920.0,NaN,-37.81188,144.88449,Western Metropolitan,6543.0


## 2.  Pré-processamento dos Dados

###2.1   Seleção de colunas







In [3]:
print(sales_data.columns)

Index(['Suburb', 'Address', 'Rooms', 'Type', 'Price', 'Method', 'SellerG',
       'Date', 'Distance', 'Postcode', 'Bedroom2', 'Bathroom', 'Car',
       'Landsize', 'BuildingArea', 'YearBuilt', 'CouncilArea', 'Lattitude',
       'Longtitude', 'Regionname', 'Propertycount'],
      dtype='object')




####   Colunas relevantes:

1. **Price:** Fundamental, pois é a base para categorizar os imóveis.
2. **Rooms:** O número de quartos é um indicador importante do tamanho e valor da propriedade.
3. **Bathroom:** O número de WCs geralmente aumenta o preço e o interesse de um imóvel.
4. **Landsize:** O tamanho do terreno é um fator relevante para o valor.
5. **BuildingArea:** Representa a área construída, outro fator crucial.
6. **Distance:** Proximidade ao centro de negócios, o que pode influenciar o preço.
8. **Car:** O número de vagas de estacionamento, importante para muitos compradores.

Do amplo número de colunas disponíveis escolhemos as acima como sendo as mais relevantes para a construção dos modelos a testar


In [5]:
# Selecionar colunas relevantes
df = sales_data[["Price", "Rooms", "Bathroom", "Landsize", "BuildingArea", "Distance", "Car"]]

df2 = sales_data[["Price", "Rooms", "Bathroom", "Landsize", "BuildingArea", "Distance", "Car", "YearBuilt"]] # Utilizado posteriormente para um segundo modelo


### 2.2 Tratar Valores Ausentes

In [6]:
# Remover linhas com valores ausentes
df = df.dropna()
df2 = df2.dropna()
# Verificar novamente se há valores ausentes
print(df.isnull().sum())


Price           0
Rooms           0
Bathroom        0
Landsize        0
BuildingArea    0
Distance        0
Car             0
dtype: int64


### 2.3 Criar Categorias Com Base no Preço



*   Preço abaixo de 600000 Acessível
*   Preço entre 600000 e 1200000 Média
*   Preço acima de 1200000 Luxuosa



In [7]:
# Função para categorizar o preço
def categorizar_preco(preco):
    if preco < 600000:
        return "Acessível"
    elif preco < 1200000:
        return "Média"
    else:
        return "Luxuosa"

# Criar a coluna "Categoria" com base no preço
df["Categoria"] = df["Price"].apply(categorizar_preco)

# Criar a coluna "Categoria" com base no preço df2
df2["Categoria"] = df2["Price"].apply(categorizar_preco)

# Remover a coluna "Price", pois não será usada como feature
df = df.drop(columns=["Price"])



## 3. Dividir dados entre treino e teste

*   80% dos dados para o modelo de treino;
*   20% dos dados para o avaliação do modelo;
*   (`random_state=42`) conjuntos fixos de treino e teste facilita identificar erros ou testar mudanças sem introduzir variações aleatórias




In [9]:
# Variáveis independentes (X) e alvo (y) com referência t1
X_t1 = df.drop(columns=["Categoria"])
y_t1 = df["Categoria"]

# Dividir os dados em treino e teste (t1)
from sklearn.model_selection import train_test_split
X_train_t1, X_test_t1, y_train_t1, y_test_t1 = train_test_split(X_t1, y_t1, test_size=0.2, random_state=42)

# Verificar os tamanhos dos conjuntos
print("Tamanho do conjunto de treino (t1):", X_train_t1.shape)
print("Tamanho do conjunto de teste (t1):", X_test_t1.shape)

Tamanho do conjunto de treino (t1): (5680, 6)
Tamanho do conjunto de teste (t1): (1421, 6)


### 4. Treinar os Modelos


*   Random Forest
*   KNN
*   KNN, com Bagging





####  4.1 ***Random Forest***, utilizando 100 árvores no modelo.

In [10]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Treinar o modelo Random Forest com os dados t1
clf_t1 = RandomForestClassifier(n_estimators=100, random_state=42)
clf_t1.fit(X_train_t1, y_train_t1)

# Fazer previsões no conjunto de treino
y_pred_train_t1 = clf_t1.predict(X_train_t1)

# Fazer previsões no conjunto de teste
y_pred_t1 = clf_t1.predict(X_test_t1)

# Avaliar o modelo no conjunto de treino
print("\n--- Resultados no Conjunto de Treino (t1) ---")
print("Matriz de confusão (treino):")
print(confusion_matrix(y_train_t1, y_pred_train_t1))

print("\nRelatório de classificação (treino):")
print(classification_report(y_train_t1, y_pred_train_t1))

accuracy_train_t1 = accuracy_score(y_train_t1, y_pred_train_t1) * 100
print(f"\nPrecisão no treino (t1): {accuracy_train_t1:.2f}%")

# Avaliar o modelo no conjunto de teste
print("\n--- Resultados no Conjunto de Teste (t1) ---")
print("Matriz de confusão (teste):")
print(confusion_matrix(y_test_t1, y_pred_t1))

print("\nRelatório de classificação (teste):")
print(classification_report(y_test_t1, y_pred_t1))

accuracy_test_t1 = accuracy_score(y_test_t1, y_pred_t1) * 100
print(f"\nPrecisão no teste (t1): {accuracy_test_t1:.2f}%")



--- Resultados no Conjunto de Treino (t1) ---
Matriz de confusão (treino):
[[1189    0    5]
 [   0 1848    1]
 [   3    0 2634]]

Relatório de classificação (treino):
              precision    recall  f1-score   support

   Acessível       1.00      1.00      1.00      1194
     Luxuosa       1.00      1.00      1.00      1849
       Média       1.00      1.00      1.00      2637

    accuracy                           1.00      5680
   macro avg       1.00      1.00      1.00      5680
weighted avg       1.00      1.00      1.00      5680


Precisão no treino (t1): 99.84%

--- Resultados no Conjunto de Teste (t1) ---
Matriz de confusão (teste):
[[197   3 100]
 [  0 332 116]
 [ 49  90 534]]

Relatório de classificação (teste):
              precision    recall  f1-score   support

   Acessível       0.80      0.66      0.72       300
     Luxuosa       0.78      0.74      0.76       448
       Média       0.71      0.79      0.75       673

    accuracy                           0.7

#### 4.1.1 Intrepertação de resultados de teste


| Verdadeiro\Predito | Acessível | Luxuosa | Média |
|--------------------|-----------|---------|-------|
| **Acessível**      | 197       | 3       | 100   |
| **Luxuosa**        | 0         | 332     | 116   |
| **Média**          | 49        | 90      | 534   |

---

**Valores na Diagonal representam as classificações corretas.(197, 332, 534)**

**Valores Fora da Diagonal Representam erros do modelo:**

* Acessível → Média (100): Muitos imóveis "Acessíveis" foram confundidos com "Média".
* Média → Luxuosa (90): Alguns imóveis "Média" foram confundidos com "Luxuosa".
* Média → Acessível (49): Alguns imóveis "Média" foram classificados como "Acessível".
* Luxuosa → Média (116): Uma quantidade significativa de imóveis "Luxuosa" foi classificada incorretamente como "Média".
---
**Precisão**:
* Das vezes que o modelo previu "Acessível", 80% estavam corretas.
*  Das vezes que o modelo previu "Luxuosa", 78% estavam corretas.
* Das vezes que o modelo previu "Média", 71% estavam corretas.

(A precisão é menor para "Média", sugerindo que o modelo confunde "Média" com outras classes mais frequentemente)

---
**Recall**(Capacidade de identificar corretamente as instâncias reais de uma classe):
* O modelo conseguiu identificar corretamente 66% dos imóveis realmente "Acessíveis"
*  O modelo conseguiu identificar corretamente 74% dos imóveis realmente "Luxuosa".
* O modelo conseguiu identificar corretamente 79% dos imóveis realmente "Média".

(O recall é baixo para "Acessível" (66%), indicando que muitos imóveis "Acessíveis" foram classificados incorretamente como outras classes.)

---
**F1-Score** (média harmônica entre precisão e recall):
* Moderada para "Acessível" (0.72)
* Boa para "Luxuosa" (0.76)
* Equilibrada para "Média" (0.75)

---
**Médias Globais**
* Macro Avg: O modelo tem desempenho médio razoável entre as classes
* Weighted Avg: O modelo tem desempenho geral consistente e ligeiramente melhor para a classe "Média"


Nota:
- **F1**- Score refere-se à media harmónica entre a precisão e o recall
- **Recall** - Mede a capacidade do modelo de identificar todas as instâncias positivas
- **Support** - Métrica simples que indica o número total de exemplos reais de uma classe
- **Precison** - Das instâncias classificadas como positivas quais são realmente positivas
- **Macro AVG** - Calcula a média simples da métrica
- **Weighted AVG** - Calcula a média poderada da métrica


#### 4.1.2 ***K-Fold Cross-Validation***

In [11]:
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

# Configurar o K-Fold
kf_t1 = KFold(n_splits=5, shuffle=True, random_state=42)

# Lista para armazenar as Precisões
accuracies_t1 = []

# Loop pelos folds
for train_index, test_index in kf_t1.split(X_t1):
    # Dividir os dados em treino e teste para o fold atual
    X_train_fold_t1, X_test_fold_t1 = X_t1.iloc[train_index], X_t1.iloc[test_index]
    y_train_fold_t1, y_test_fold_t1 = y_t1.iloc[train_index], y_t1.iloc[test_index]

    # Usar o modelo já configurado (clf_t1)
    clf_t1.fit(X_train_fold_t1, y_train_fold_t1)
    y_pred_fold_t1 = clf_t1.predict(X_test_fold_t1)

    # Avaliar a Precisão no fold atual
    acc_t1 = accuracy_score(y_test_fold_t1, y_pred_fold_t1)
    accuracies_t1.append(acc_t1 * 100)

# Resultados do k-fold
print("Precisão em cada fold(t1) (%):", [f"{acc:.2f}%" for acc in accuracies_t1])
print("Precisão Média(t1) (%):", f"{sum(accuracies_t1) / len(accuracies_t1):.2f}%")
print("Desvio Padrão da Precisão(t1) (%):", f"{pd.Series(accuracies_t1).std():.2f}%")

Precisão em cada fold(t1) (%): ['74.67%', '71.20%', '72.11%', '72.68%', '72.04%']
Precisão Média(t1) (%): 72.54%
Desvio Padrão da Precisão(t1) (%): 1.30%


#### 4.2 ***KNN (K-Nearest Neighbors)***, com 5 vizinhos

In [12]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Configurar o modelo KNN com 5 vizinhos
knn_t1 = KNeighborsClassifier(n_neighbors=5)

# Treinar o modelo
knn_t1.fit(X_train_t1, y_train_t1)

# Fazer previsões no conjunto de treino
y_pred_train_knn_t1 = knn_t1.predict(X_train_t1)

# Fazer previsões no conjunto de teste
y_pred_test_knn_t1 = knn_t1.predict(X_test_t1)

# Avaliar o modelo no conjunto de treino
print("\n--- Resultados no Conjunto de Treino (KNN, t1) ---")
print("Matriz de confusão (treino):")
print(confusion_matrix(y_train_t1, y_pred_train_knn_t1))

print("\nRelatório de classificação (treino):")
print(classification_report(y_train_t1, y_pred_train_knn_t1))

accuracy_train_knn_t1 = accuracy_score(y_train_t1, y_pred_train_knn_t1) * 100
print(f"\nPrecisão no treino (KNN, t1): {accuracy_train_knn_t1:.2f}%")

# Avaliar o modelo no conjunto de teste
print("\n--- Resultados no Conjunto de Teste (KNN, t1) ---")
print("Matriz de confusão (teste):")
print(confusion_matrix(y_test_t1, y_pred_test_knn_t1))

print("\nRelatório de classificação (teste):")
print(classification_report(y_test_t1, y_pred_test_knn_t1))

accuracy_test_knn_t1 = accuracy_score(y_test_t1, y_pred_test_knn_t1) * 100
print(f"\nPrecisão no teste (KNN, t1): {accuracy_test_knn_t1:.2f}%")



--- Resultados no Conjunto de Treino (KNN, t1) ---
Matriz de confusão (treino):
[[ 842   43  309]
 [  33 1379  437]
 [ 207  419 2011]]

Relatório de classificação (treino):
              precision    recall  f1-score   support

   Acessível       0.78      0.71      0.74      1194
     Luxuosa       0.75      0.75      0.75      1849
       Média       0.73      0.76      0.75      2637

    accuracy                           0.75      5680
   macro avg       0.75      0.74      0.74      5680
weighted avg       0.75      0.75      0.75      5680


Precisão no treino (KNN, t1): 74.51%

--- Resultados no Conjunto de Teste (KNN, t1) ---
Matriz de confusão (teste):
[[188  14  98]
 [  9 278 161]
 [ 83 158 432]]

Relatório de classificação (teste):
              precision    recall  f1-score   support

   Acessível       0.67      0.63      0.65       300
     Luxuosa       0.62      0.62      0.62       448
       Média       0.63      0.64      0.63       673

    accuracy               

#### 4.2.1 ***K-Fold Cross-Validation***

In [13]:
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# Configurar o K-Fold
kf_knn_t1 = KFold(n_splits=5, shuffle=True, random_state=42)

# Lista para armazenar as Precisões
accuracies_knn_t1 = []

# Loop pelos folds
for train_index, test_index in kf_knn_t1.split(X_t1):
    # Dividir os dados em treino e teste para o fold atual
    X_train_fold_knn, X_test_fold_knn = X_t1.iloc[train_index], X_t1.iloc[test_index]
    y_train_fold_knn, y_test_fold_knn = y_t1.iloc[train_index], y_t1.iloc[test_index]

    # Configurar e treinar o modelo KNN
    knn_fold_t1 = KNeighborsClassifier(n_neighbors=5)
    knn_fold_t1.fit(X_train_fold_knn, y_train_fold_knn)

    # Fazer previsões
    y_pred_fold_knn = knn_fold_t1.predict(X_test_fold_knn)

    # Avaliar a precisão no fold atual
    acc_knn_t1 = accuracy_score(y_test_fold_knn, y_pred_fold_knn)
    accuracies_knn_t1.append(acc_knn_t1 * 100)

# Resultados do K-Fold
print("\nPrecisão em cada fold (KNN)(t1) (%):", [f"{acc:.2f}%" for acc in accuracies_knn_t1])
print("Precisão Média (KNN) (%):", f"{sum(accuracies_knn_t1) / len(accuracies_knn_t1):.2f}%")
print("Desvio Padrão da precisão (KNN)(t1) (%):", f"{pd.Series(accuracies_knn_t1).std():.2f}%")



Precisão em cada fold (KNN)(t1) (%): ['63.19%', '62.75%', '64.08%', '63.66%', '62.04%']
Precisão Média (KNN) (%): 63.15%
Desvio Padrão da precisão (KNN)(t1) (%): 0.79%


#### 4.3 ***KNN (K-Nearest Neighbors)***, com 5 vizinhos e utilizando um bagging de 50 instâncias

In [14]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

# Configurar o modelo base (KNN)
base_model_knn_t1 = KNeighborsClassifier(n_neighbors=5)

# Configurar o BaggingClassifier
bagging_knn_t1 = BaggingClassifier(estimator=base_model_knn_t1, n_estimators=50, random_state=42)

# Treinar o modelo com os dados t1
bagging_knn_t1.fit(X_train_t1, y_train_t1)

# Fazer previsões no conjunto de treino
y_pred_train_bagging_knn_t1 = bagging_knn_t1.predict(X_train_t1)

# Fazer previsões no conjunto de teste
y_pred_test_bagging_knn_t1 = bagging_knn_t1.predict(X_test_t1)

# Avaliar o modelo no conjunto de treino
print("\n--- Resultados no Conjunto de Treino - Bagging (KNN, t1) ---")
print("Matriz de Confusão (treino):")
print(confusion_matrix(y_train_t1, y_pred_train_bagging_knn_t1))

print("\nRelatório de Classificação (treino):")
print(classification_report(y_train_t1, y_pred_train_bagging_knn_t1))

accuracy_train_bagging_knn_t1 = accuracy_score(y_train_t1, y_pred_train_bagging_knn_t1) * 100
print(f"\nPrecisão no treino (Bagging KNN, t1): {accuracy_train_bagging_knn_t1:.2f}%")

# Avaliar o modelo no conjunto de teste
print("\n--- Resultados no Conjunto de Teste - Bagging (KNN, t1) ---")
print("Matriz de Confusão (teste):")
print(confusion_matrix(y_test_t1, y_pred_test_bagging_knn_t1))

print("\nRelatório de Classificação (teste):")
print(classification_report(y_test_t1, y_pred_test_bagging_knn_t1))

accuracy_test_bagging_knn_t1 = accuracy_score(y_test_t1, y_pred_test_bagging_knn_t1) * 100
print(f"\nPrecisão no teste (Bagging KNN, t1): {accuracy_test_bagging_knn_t1:.2f}%")



--- Resultados no Conjunto de Treino - Bagging (KNN, t1) ---
Matriz de Confusão (treino):
[[ 815   29  350]
 [  18 1375  456]
 [ 160  361 2116]]

Relatório de Classificação (treino):
              precision    recall  f1-score   support

   Acessível       0.82      0.68      0.75      1194
     Luxuosa       0.78      0.74      0.76      1849
       Média       0.72      0.80      0.76      2637

    accuracy                           0.76      5680
   macro avg       0.77      0.74      0.76      5680
weighted avg       0.76      0.76      0.76      5680


Precisão no treino (Bagging KNN, t1): 75.81%

--- Resultados no Conjunto de Teste - Bagging (KNN, t1) ---
Matriz de Confusão (teste):
[[173  13 114]
 [  7 268 173]
 [ 64 143 466]]

Relatório de Classificação (teste):
              precision    recall  f1-score   support

   Acessível       0.71      0.58      0.64       300
     Luxuosa       0.63      0.60      0.61       448
       Média       0.62      0.69      0.65       673


#### 4.3.1 K-Fold Cross-Validation

In [15]:
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

# Configurar o K-Fold
kf_bagging_t1 = KFold(n_splits=5, shuffle=True, random_state=42)

# Lista para armazenar Precisão
accuracies_bagging_knn_t1 = []

# Loop pelos folds
for train_index, test_index in kf_bagging_t1.split(X_t1):
    # Dividir os dados em treino e teste para o fold atual
    X_train_fold_bagging, X_test_fold_bagging = X_t1.iloc[train_index], X_t1.iloc[test_index]
    y_train_fold_bagging, y_test_fold_bagging = y_t1.iloc[train_index], y_t1.iloc[test_index]

    # Configurar e treinar o modelo KNN com Bagging
    bagging_knn_fold = BaggingClassifier(
        estimator=KNeighborsClassifier(n_neighbors=5),
        n_estimators=50,
        random_state=42
    )
    bagging_knn_fold.fit(X_train_fold_bagging, y_train_fold_bagging)

    # Fazer previsões
    y_pred_fold_bagging = bagging_knn_fold.predict(X_test_fold_bagging)

    # Avaliar a precisão no fold atual
    acc_bagging_knn_t1 = accuracy_score(y_test_fold_bagging, y_pred_fold_bagging)
    accuracies_bagging_knn_t1.append(acc_bagging_knn_t1 * 100)  # Convertendo para percentagem

# Resultados do K-Fold
print("\nPrecisão em cada fold (Bagging KNN)(t1) (%):", [f"{acc:.2f}%" for acc in accuracies_bagging_knn_t1])
print("Precisão Média (Bagging KNN)(t1) (%):", f"{sum(accuracies_bagging_knn_t1) / len(accuracies_bagging_knn_t1):.2f}%")
print("Desvio Padrão da Precisão (Bagging KNN)(t1) (%):", f"{pd.Series(accuracies_bagging_knn_t1).std():.2f}%")



Precisão em cada fold (Bagging KNN)(t1) (%): ['63.69%', '64.93%', '65.63%', '65.56%', '64.37%']
Precisão Média (Bagging KNN)(t1) (%): 64.84%
Desvio Padrão da Precisão (Bagging KNN)(t1) (%): 0.82%


#### 4.4 Intrepertação de Resultados ***KNN VS KNN-BAGGING*** (Dados teste)



**Comparação por classe**
* 1. Classe Acessível


| **Métrica**   | **KNN Simples** | **Bagging (KNN)** | **Diferença** |
|---------------|-----------------|-------------------|---------------|
| **Precisão**  | 0.67            | 0.71              | +0.04         |
| **Recall**    | 0.63            | 0.58              | -0.05         |
| **F1-Score**  | 0.65            | 0.64              | -0.01         |

* O Bagging melhora a precisão para "Acessível" (+4%), indicando que ele cometeu menos erros ao prever essa classe.
* No entanto, o recall caiu (-5%), o que significa que o modelo perdeu mais instâncias reais dessa classe.

---

* 2. Classe Luxuosa

| **Métrica**   | **KNN Simples** | **Bagging (KNN)** | **Diferença** |
|---------------|-----------------|-------------------|---------------|
| **Precisão**  | 0.62            | 0.63              | +0.01         |
| **Recall**    | 0.62            | 0.60              | -0.02         |
| **F1-Score**  | 0.62            | 0.61              | -0.01         |

* Pequena melhora na precisão (+1%) com o Bagging, mas o recall reduziu ligeiramente (-2%).
* O desempenho geral para "Luxuosa" é semelhante entre os dois modelos
---

* 3. Classe Media

| **Métrica**   | **KNN Simples** | **Bagging (KNN)** | **Diferença** |
|---------------|-----------------|-------------------|---------------|
| **Precisão**  | 0.63            | 0.62              | -0.01         |
| **Recall**    | 0.64            | 0.69              | +0.05         |
| **F1-Score**  | 0.63            | 0.65              | +0.02         |

* O Bagging melhora significativamente o recall para "Média" (+5%), o que significa que ele identificou mais imóveis dessa classe corretamente.
* O F1-Score também aumentou (+2%), mostrando um melhor equilíbrio entre precisão e recall.
---

* 4. Métricas Globais

| **Métrica**      | **KNN Simples**     | **Bagging (KNN)**    | **Diferença**          |
|-------------------|---------------------|----------------------|------------------------|
| **Acurácia**      | 0.63               | 0.64                | +0.01                 |
| **Macro Avg**     | 0.64/0.63/0.63     | 0.65/0.62/0.63      | +0.01/-0.01/0.00      |
| **Weighted Avg**  | 0.63/0.63/0.63     | 0.64/0.64/0.64      | +0.01/+0.01/+0.01     |

**Macro Avg:**
* A precisão média (macro avg) melhorou ligeiramente com o Bagging.
* O recall médio caiu um pouco.

**Weighted Avg:**
* Todas as métricas ponderadas tiveram um pequeno aumento, refletindo uma ligeira melhoria no desempenho geral.

---

**No geral podemos dizer que o bagging melhorou o desempenho do modelo ajudando a suavizar as flutuações no desempenho e tornando o modelo mais robusto a variações nos dados.**


### 5. Criação de novos atributos

Apesar dos diferentes métodos utilizados os resultados não foram os melhores, por isso tivemos a necessidade de voltar a necessidade de criar novos atributos e voltar a treinar o modelo.

**Novos Atributos**:
* Preço por área
* Idade da propriedade
* Proximidade ao centro de negócios




In [16]:
#Preço por área
df2['preco_por_m2'] = df2['Price'] / df2['Landsize']

#Idade da propriedade
from datetime import datetime
df2['idade_propriedade'] = datetime.now().year - df2['YearBuilt']

#Distância ao centro
df2['distancia_centro'] = df2['Distance']



In [17]:
# Remover linhas com valores ausentes
df2 = df2.dropna()


# Verificar novamente se há valores ausentes
print(df2.isnull().sum())

Price                0
Rooms                0
Bathroom             0
Landsize             0
BuildingArea         0
Distance             0
Car                  0
YearBuilt            0
Categoria            0
preco_por_m2         0
idade_propriedade    0
distancia_centro     0
dtype: int64


Divisões por 0, criam valores infinitos e para lidar com eles optamos por eliminar essas linhas

In [18]:
# Selecionar apenas colunas numéricas
numerical_cols = df2.select_dtypes(include=[np.number])

# Substituir valores infinitos por NaN nas colunas numéricas
df2[numerical_cols.columns] = df2[numerical_cols.columns].replace([np.inf, -np.inf], np.nan)

# Remover linhas com NaN em qualquer coluna
df2.dropna(inplace=True)

# Verificar novamente se há valores infinitos
print("Existem valores infinitos após o tratamento?")
print(np.isinf(df2[numerical_cols.columns]).any())

Existem valores infinitos após o tratamento?
Price                False
Rooms                False
Bathroom             False
Landsize             False
BuildingArea         False
Distance             False
Car                  False
YearBuilt            False
preco_por_m2         False
idade_propriedade    False
distancia_centro     False
dtype: bool


#### 6.  Treino do novo modelo


In [19]:
# Selecionar as variáveis independentes (X_t2) e a variável alvo (y_t2)
X_t2 = df2[['Rooms', 'Bathroom', 'Landsize', 'BuildingArea', 'Distance',
            'Car', 'preco_por_m2', 'idade_propriedade', 'distancia_centro']]
y_t2 = df2['Categoria']

# Dividir os dados em treino e teste (t2)
from sklearn.model_selection import train_test_split
X_train_t2, X_test_t2, y_train_t2, y_test_t2 = train_test_split(X_t2, y_t2, test_size=0.2, random_state=42)

# Verificar os tamanhos dos conjuntos
print("Tamanho do conjunto de treino (t2):", X_train_t2.shape)
print("Tamanho do conjunto de teste (t2):", X_test_t2.shape)

Tamanho do conjunto de treino (t2): (4652, 9)
Tamanho do conjunto de teste (t2): (1163, 9)


7.  ***Random Forest*** com novos dados

In [20]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Configurar o modelo Random Forest
rf_t2 = RandomForestClassifier(
    n_estimators=100,  # Número de árvores
    random_state=42    # Para poder ser reproduzido
)

# Treinar o modelo com os dados t2
rf_t2.fit(X_train_t2, y_train_t2)

# Fazer previsões no conjunto de treino
y_pred_train_t2 = rf_t2.predict(X_train_t2)

# Fazer previsões no conjunto de teste
y_pred_test_t2 = rf_t2.predict(X_test_t2)

# Avaliar o modelo no conjunto de treino
print("\n--- Resultados no Conjunto de Treino (t2) ---")
print("Matriz de Confusão (treino):")
print(confusion_matrix(y_train_t2, y_pred_train_t2))

print("\nRelatório de Classificação (treino):")
print(classification_report(y_train_t2, y_pred_train_t2))

accuracy_train_t2 = accuracy_score(y_train_t2, y_pred_train_t2) * 100
print(f"\nPrecisão no treino (t2): {accuracy_train_t2:.2f}%")

# Avaliar o modelo no conjunto de teste
print("\n--- Resultados no Conjunto de Teste (t2) ---")
print("Matriz de Confusão (teste):")
print(confusion_matrix(y_test_t2, y_pred_test_t2))

print("\nRelatório de Classificação (teste):")
print(classification_report(y_test_t2, y_pred_test_t2))

accuracy_test_t2 = accuracy_score(y_test_t2, y_pred_test_t2) * 100
print(f"\nPrecisão no teste (t2): {accuracy_test_t2:.2f}%")



--- Resultados no Conjunto de Treino (t2) ---
Matriz de Confusão (treino):
[[ 684    0    0]
 [   0 1729    0]
 [   0    0 2239]]

Relatório de Classificação (treino):
              precision    recall  f1-score   support

   Acessível       1.00      1.00      1.00       684
     Luxuosa       1.00      1.00      1.00      1729
       Média       1.00      1.00      1.00      2239

    accuracy                           1.00      4652
   macro avg       1.00      1.00      1.00      4652
weighted avg       1.00      1.00      1.00      4652


Precisão no treino (t2): 100.00%

--- Resultados no Conjunto de Teste (t2) ---
Matriz de Confusão (teste):
[[136   0  24]
 [  0 398  34]
 [ 16  21 534]]

Relatório de Classificação (teste):
              precision    recall  f1-score   support

   Acessível       0.89      0.85      0.87       160
     Luxuosa       0.95      0.92      0.94       432
       Média       0.90      0.94      0.92       571

    accuracy                           0.

#### 7.1 ***K-Fold*** Cross-Validation

In [21]:
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Configurar o K-Fold
kf_t2 = KFold(n_splits=5, shuffle=True, random_state=42)

# Lista para armazenar as precisões
accuracies_t2 = []

# Loop pelos folds
for train_index, test_index in kf_t2.split(X_t2):
    # Dividir os dados em treino e teste para o fold atual
    X_train_fold_t2, X_test_fold_t2 = X_t2.iloc[train_index], X_t2.iloc[test_index]
    y_train_fold_t2, y_test_fold_t2 = y_t2.iloc[train_index], y_t2.iloc[test_index]

    # Configurar o modelo Random Forest
    rf_fold_t2 = RandomForestClassifier(
        n_estimators=100,
        random_state=42
    )

    # Treinar o modelo
    rf_fold_t2.fit(X_train_fold_t2, y_train_fold_t2)

    # Fazer previsões
    y_pred_fold_t2 = rf_fold_t2.predict(X_test_fold_t2)

    # Avaliar a previsão no fold atual
    acc_fold_t2 = accuracy_score(y_test_fold_t2, y_pred_fold_t2)
    accuracies_t2.append(acc_fold_t2 * 100)  # Convertendo para percentagem

# Resultados finais do K-Fold
print("\nPrecisão em cada fold (t2) (%):", [f"{acc:.2f}%" for acc in accuracies_t2])
print("Precisão Média (t2) (%):", f"{sum(accuracies_t2) / len(accuracies_t2):.2f}%")
print("Desvio Padrão da Precisão (t2) (%):", f"{pd.Series(accuracies_t2).std():.2f}%")



Precisão em cada fold (t2) (%): ['91.40%', '92.26%', '89.77%', '91.83%', '90.03%']
Precisão Média (t2) (%): 91.06%
Desvio Padrão da Precisão (t2) (%): 1.11%


### 7.2  Análise de Resultados ***Random Forest*** Atributos antigos VS Novos (dados teste)

| **Classe**   | **Precisão (Antigos)** | **Precisão (Novos)** | **Diferença** | **Recall (Antigos)** | **Recall (Novos)** | **Diferença** | **F1-Score (Antigos)** | **F1-Score (Novos)** | **Diferença** |
|--------------|-------------------------|-----------------------|---------------|-----------------------|---------------------|---------------|------------------------|-----------------------|---------------|
| **Acessível**| 0.80                    | 0.89                  | +0.09         | 0.66                  | 0.85                | +0.19         | 0.72                   | 0.87                  | +0.15         |
| **Luxuosa**  | 0.78                    | 0.95                  | +0.17         | 0.74                  | 0.92                | +0.18         | 0.76                   | 0.94                  | +0.18         |
| **Média**    | 0.71                    | 0.90                  | +0.19         | 0.79                  | 0.94                | +0.15         | 0.75                   | 0.92                  | +0.17         |



**Análise**:

* As métricas (precisão, recall e F1-score) melhoraram significativamente para todas as classes.
* O maior ganho foi na classe Luxuosa, com +17% de precisão e +18% de F1-score, indicando que os novos atributos ajudaram a capturar melhor as diferenças entre as classes.


**Macro Média** (Macro Avg)

| **Métrica** | **Atributos Antigos** | **Atributos Novos** | **Diferença** |
|-------------|------------------------|---------------------|---------------|
| **Precisão**| 0.76                   | 0.92                | +0.16         |
| **Recall**  | 0.73                   | 0.90                | +0.17         |
| **F1-Score**| 0.74                   | 0.91                | +0.17         |
**Diferenças:** +16% em Precisão, +17% em Recall, +17% em F1-Score.



**Média Ponderada** (Weighted Avg)

| **Métrica** | **Atributos Antigos** | **Atributos Novos** | **Diferença** |
|-------------|------------------------|---------------------|---------------|
| **Precisão**| 0.75                   | 0.92                | +0.17         |
| **Recall**  | 0.75                   | 0.92                | +0.17         |
| **F1-Score**| 0.75                   | 0.92                | +0.17         |
**Diferenças:** +17% em Precisão, Recall e F1-Score.


----

| **Classe**   | **TP (Antigos)** | **TP (Novos)** | **Diferença** |
|--------------|------------------|----------------|---------------|
| **Acessível**| 197              | 136            | -61           |
| **Luxuosa**  | 332              | 398            | +66           |
| **Média**    | 534              | 534            | 0             |

**Análise**:

* Apesar da diferença no tamanho do conjunto de teste (1421 vs 1163), o modelo com os novos atributos:
* Reduziu significativamente os falsos positivos e falsos negativos.
* Apresentou uma melhor classificação nas classes "Luxuosa" e "Média".

**Conclusão**: Os novos atributos (como preco_por_m2, idade_propriedade, etc.) melhoraram consideravelmente o desempenho do modelo em todas as métricas.
A precisão, recall e F1-score aumentaram significativamente, especialmente para as classes "Luxuosa" e "Média".
O modelo agora generaliza melhor, resultando numa precisão global de 92%.

#### 7.3 ***KNN (K-Nearest Neighbors)***, com 5 vizinhos e utilizando um bagging de 50 instâncias

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Configurar o modelo base (KNN)
base_model_knn_t2 = KNeighborsClassifier(n_neighbors=5)

# Configurar o BaggingClassifier
bagging_knn_t2 = BaggingClassifier(
    estimator=base_model_knn_t2,  # Modelo base (KNN)
    n_estimators=50,              # Número de modelos no ensemble
    random_state=42               # Para reprodutibilidade
)

# Treinar o modelo com os dados t2
bagging_knn_t2.fit(X_train_t2, y_train_t2)

# Fazer previsões no conjunto de treino
y_pred_train_bagging_knn_t2 = bagging_knn_t2.predict(X_train_t2)

# Fazer previsões no conjunto de teste
y_pred_test_bagging_knn_t2 = bagging_knn_t2.predict(X_test_t2)

# Avaliar o modelo no conjunto de treino
print("\n--- Resultados no Conjunto de Treino - Bagging (KNN, t2) ---")
print("Matriz de Confusão (treino):")
print(confusion_matrix(y_train_t2, y_pred_train_bagging_knn_t2))

print("\nRelatório de Classificação (treino):")
print(classification_report(y_train_t2, y_pred_train_bagging_knn_t2))

accuracy_train_bagging_knn_t2 = accuracy_score(y_train_t2, y_pred_train_bagging_knn_t2) * 100
print(f"\nPrecisão no treino (Bagging KNN, t2): {accuracy_train_bagging_knn_t2:.2f}%")

# Avaliar o modelo no conjunto de teste
print("\n--- Resultados no Conjunto de Teste - Bagging (KNN, t2) ---")
print("Matriz de Confusão (teste):")
print(confusion_matrix(y_test_t2, y_pred_test_bagging_knn_t2))

print("\nRelatório de Classificação (teste):")
print(classification_report(y_test_t2, y_pred_test_bagging_knn_t2))

accuracy_test_bagging_knn_t2 = accuracy_score(y_test_t2, y_pred_test_bagging_knn_t2) * 100
print(f"\nPrecisão no teste (Bagging KNN, t2): {accuracy_test_bagging_knn_t2:.2f}%")


### Análise de Resultados ***KNN com Bagging*** Atributos antigos VS Novos



| **Métrica**       | **Acessível (Antigo)** | **Acessível (Novo)** | **Diferença** | **Luxuosa (Antigo)** | **Luxuosa (Novo)** | **Diferença** | **Média (Antigo)** | **Média (Novo)** | **Diferença** |
|--------------------|------------------------|-----------------------|---------------|-----------------------|--------------------|---------------|--------------------|-----------------|---------------|
| **Precisão**       | 0.71                  | 0.90                 | +0.19         | 0.63                 | 0.94              | +0.31         | 0.62              | 0.91            | +0.29         |
| **Recall**         | 0.58                  | 0.84                 | +0.26         | 0.60                 | 0.94              | +0.34         | 0.69              | 0.93            | +0.24         |
| **F1-Score**       | 0.64                  | 0.87                 | +0.23         | 0.61                 | 0.94              | +0.33         | 0.65              | 0.92            | +0.27         |
| **Acurácia Geral** | 0.64                  | 0.92                 | +0.28         |                       |                    |               |                    |                 |               |
| **Macro Avg**      | 0.65/0.62/0.63        | 0.92/0.90/0.91       | +0.27/+0.28/+0.28 |                     |                    |               |                    |                 |               |
| **Weighted Avg**   | 0.64/0.64/0.64        | 0.92/0.92/0.92       | +0.28/+0.28/+0.28 |                     |                    |               |                    |                 |               |


**Análise**: A inclusão de novos atributos aumentou significativamente a capacidade do modelo de classificar corretamente as observações, sugerindo que os atributos adicionados capturam informações mais relevantes sobre as classes.


***K-Fold*** Cross-Validation

In [ ]:
from sklearn.model_selection import KFold
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# Configurar o K-Fold
kf_t2 = KFold(n_splits=5, shuffle=True, random_state=42)

# Lista para armazenar as acurácias
accuracies_bagging_knn_t2 = []

# Loop pelos folds
for fold, (train_index, test_index) in enumerate(kf_t2.split(X_t2)):
    # Dividir os dados em treino e teste para o fold atual
    X_train_fold_t2, X_test_fold_t2 = X_t2.iloc[train_index], X_t2.iloc[test_index]
    y_train_fold_t2, y_test_fold_t2 = y_t2.iloc[train_index], y_t2.iloc[test_index]

    # Configurar o modelo KNN com Bagging
    bagging_knn_fold_t2 = BaggingClassifier(
        estimator=KNeighborsClassifier(n_neighbors=5),
        n_estimators=50,
        random_state=42
    )

    # Treinar o modelo
    bagging_knn_fold_t2.fit(X_train_fold_t2, y_train_fold_t2)

    # Fazer previsões
    y_pred_fold_t2 = bagging_knn_fold_t2.predict(X_test_fold_t2)

    # Avaliar a Precisão no fold atual
    acc_fold_t2 = accuracy_score(y_test_fold_t2, y_pred_fold_t2)
    accuracies_bagging_knn_t2.append(acc_fold_t2 * 100)  # Convertendo para percentagem

# Resultados finais do K-Fold
print("\nPrecisão em cada fold (%):", [f"{acc:.2f}%" for acc in accuracies_bagging_knn_t2])
print("Precisão Média (%):", f"{sum(accuracies_bagging_knn_t2) / len(accuracies_bagging_knn_t2):.2f}%")
print("Desvio Padrão da Precisão (%):", f"{pd.Series(accuracies_bagging_knn_t2).std():.2f}%")




Precisão em cada fold (%): ['92.00%', '92.52%', '92.61%', '92.78%', '94.07%']
Precisão Média (%): 92.79%
Desvio Padrão da Precisão (%): 0.77%


### 8.  Avaliação final



#### 8.1 Comparação de resultados


| Modelo            | Conjunto | Precisão Fixa (%) | F1-Score Fixo | Precisão Média K-Fold (%) | Desvio Padrão K-Fold (%) |
|-------------------|----------|-------------------|---------------|---------------------------|--------------------------|
| Random Forest     | t1       | 75.00            | 0.74          | 72.54                    | 1.30                    |
| KNN               | t1       | 63.19            | 0.63          | 63.15                    | 0.79                    |
| Bagging (KNN)     | t1       | 63.83            | 0.64          | 64.84                    | 0.82                    |
| Random Forest     | t2       | 91.83            | 0.92          | 91.06                    | 1.11                    |
| Bagging (KNN)     | t2       | 91.92            | 0.92          | 92.79                    | 0.77                    |


----


**1. Desempenho Geral do Conjunto t2 é Superior ao t1**
Todos os modelos treinados com o conjunto t2 apresentaram melhores métricas de desempenho em comparação com os modelos do conjunto t1.


Precisão Fixa e Média no K-Fold em t2 estão consistentemente acima de 91%, enquanto no conjunto t1, a melhor precisão é de 75% (Random Forest).


**2. Random Forest Sobressai no Conjunto t2**

| Métrica              | Valor       |
|----------------------|-------------|
| Precisão Fixa        | 91.83%      |
| Precisão Média K-Fold| 91.06%      |
| F1-Score Fixo        | 0.92        |

Embora o Bagging (KNN, t2) tenha uma ligeira vantagem na preecisão média no K-Fold (92.79%), o Random Forest oferece uma combinação robusta de desempenho fixo e estabilidade.


**4. Modelos no t1 São Inferiores**

Mesmo o melhor modelo no conjunto t1 (Random Forest) apresentou:

| Métrica              | Valor       |
|----------------------|-------------|
| Precisão Fixa        | 75%         |
| Precisão Média K-Fold| 72.54%      |
| F1-Score Fixo        | 0.74        |


Isso reforça que o conjunto t1 possui menos informações relevantes para os modelos.

**5. Escolha do Modelo Final**

Com base nesses resultados, o modelo final escolhido foi o Random Forest (t2),por ser mais rápido para deduções, enquanto Bagging (KNN) requer mais computação devido ao ensemble de vizinhos mais próximos.

###**Conclusão**
- O conjunto de dados t2 deve ser usado como base para o modelo final.
- O modelo Random Forest (t2) é a escolha principal devido ao equilíbrio entre desempenho fixo e estabilidade.
